This notebok takes the excel list of all stamps, copares to current hiking journal and sets visited_by_me flag

Also looks for all loaded images in the google drive (box images and selfies) and adds these flags too

In [1]:
import os

print("PYTHONPATH environment variable:")
print(os.environ.get('PYTHONPATH'))


PYTHONPATH environment variable:
./source


In [2]:
import os
import sys
from pathlib import Path

print("Working directory:", Path().resolve())
print("PYTHONPATH env var:", os.environ.get("PYTHONPATH"))
print("sys.path:")
for p in sys.path:
    print(" -", p)


Working directory: /Users/olehbakumenko/Desktop/oleg/Workspaces/PytonWorkspace/harz_tsp/source/notebooks
PYTHONPATH env var: ./source
sys.path:
 - /Users/olehbakumenko/Desktop/oleg/Workspaces/PytonWorkspace/harz_tsp/source/notebooks/source
 - /Users/olehbakumenko/miniforge3/envs/harz_tsp/lib/python310.zip
 - /Users/olehbakumenko/miniforge3/envs/harz_tsp/lib/python3.10
 - /Users/olehbakumenko/miniforge3/envs/harz_tsp/lib/python3.10/lib-dynload
 - 
 - /Users/olehbakumenko/miniforge3/envs/harz_tsp/lib/python3.10/site-packages


In [6]:
import utils.utilities as uu


ModuleNotFoundError: No module named 'utils'

In [3]:
import pandas as pd
import os
import sys
import gpxpy.gpx
import gpxpy
import glob
from pathlib import Path
import source.utils.utilities as uu
path_to_source = uu.get_root_path()

ModuleNotFoundError: No module named 'source'

In [4]:
def transform_stamp_name_to_int(name):
    """
    This function takes "HWN002 Scharfenstein" and should return 2
    """
    try:
        number_str = name.split("HWN")[1][:3]
    except:
        number_str = -1
    return int(number_str)

export to excel

In [ ]:
path_to_gpx = Path(path_to_source, 'gpx')
all_stamps = pd.read_excel(Path(
    path_to_source, 'excel', 'all_stamps.xlsx'))

if something does not make sence, most likely next cell was just copy pasted

In [ ]:
# Download the correct list from the google drive called Hiking_journal.xls
my_trips_df = pd.read_excel(os.path.join(
    path_to_source, 'my_trips', 'Hiking_journal.xlsx'))

my_trips_df = my_trips_df[my_trips_df['Nr. stamp'].notna()]

# set nr stapms to integer
my_trips_df['Nr. stamp'] = my_trips_df['Nr. stamp'].astype(int)

all_my_visited_stamps = my_trips_df["Place name"].tolist()
all_my_visited_stamps = [transform_stamp_name_to_int(
    name) for name in all_my_visited_stamps]

In [6]:
all_stamps["visited_by_me"] = 0

for i, row in all_stamps.iterrows():
    if row["number"] in all_my_visited_stamps:
        all_stamps.loc[i, "visited_by_me"] = 1
    else:
        all_stamps.loc[i, "visited_by_me"] = 0

all_stamps["visited_by_me"] = all_stamps["visited_by_me"].astype(int)

In [7]:
print(f"Number stamp visited by me: {all_stamps['visited_by_me'].sum()}")

Number stamp visited by me: 72


In [8]:
image_folder =  Path(r"/Users/olehbakumenko/Downloads/Harz/My trips/images")
jpg_files = glob.glob(os.path.join(image_folder, '*.jpg'))
only_boxes_imgs = [Path(file) for file in jpg_files if "_box" in file and "So" not in file]
only_selfies_imgs= [Path(file)  for file in jpg_files if "_selfie" in file and "So" not in file]

print(len(only_boxes_imgs))
print(len(only_selfies_imgs))

only_boxes_numbers = [int(file.name.split("_")[0]) for file in only_boxes_imgs]
only_selfies_numbers = [int(file.name.split("_")[0]) for file in only_selfies_imgs]

67
54


In [9]:
all_stamps["number"]


0        1
1        2
2        3
3        4
4        5
      ... 
217    218
218    219
219    220
220    221
221    222
Name: number, Length: 222, dtype: int64

In [10]:
all_stamps["box_image"] = 0

for i, row in all_stamps.iterrows():
    if row["number"] in only_boxes_numbers:
        all_stamps.loc[i, "box_image"] = 1
    else:
        all_stamps.loc[i, "box_image"] = 0

all_stamps["selfie_image"] = 0
for i, row in all_stamps.iterrows():
    if row["number"] in only_selfies_numbers:
        all_stamps.loc[i, "selfie_image"] = 1
    else:
        all_stamps.loc[i, "selfie_image"] = 0


In [11]:
for i, row in all_stamps.iterrows():
    if row["box_image"] ==1 and not row["selfie_image"] == 1:
        all_stamps.loc[i, "selfie_image"] = "no_selfie"
        
for i, row in all_stamps.iterrows():
    if row["visited_by_me"] ==1 and not row["box_image"] == 1:
        all_stamps.loc[i, "box_image"] = "no_box"

/var/folders/87/_jvy15c55mx0cfd6dqw9v64r0000gn/T/ipykernel_9463/224945285.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'no_selfie' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_stamps.loc[i, "selfie_image"] = "no_selfie"
/var/folders/87/_jvy15c55mx0cfd6dqw9v64r0000gn/T/ipykernel_9463/224945285.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'no_box' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_stamps.loc[i, "box_image"] = "no_box"


In [12]:
all_stamps

,number,name,latitude,longitude,elevation,description,comment,hexe,steiger,grenze,goethe,visited_by_me,box_image,selfie_image
0,1,HWN001 Eckertalsperre,51.841649,10.579976,555,Eckertalsperre (Staumauer),Eckertalsperre (Staumauer),0,0,1,0,1,1,1
1,2,HWN002 Scharfenstein,51.830168,10.602774,635,Scharfenstein (Rangerstation),Scharfenstein (Rangerstation),0,0,1,0,0,0,0
2,3,HWN003 Am Kruzifix,51.844150,10.611610,567,Am Kruzifix (Schutzhütte),Am Kruzifix (Schutzhütte),0,0,1,0,0,0,0
3,4,HWN004 Taubenklippe,51.862558,10.618166,555,Taubenklippe,Taubenklippe,0,0,1,0,0,0,0
4,5,HWN005 Froschfelsen,51.851915,10.654570,523,Froschfelsen,Froschfelsen,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,218,HWN218 Neustädter Talsperre,51.578883,10.863333,460,Neustädter Talsperre,Neustädter Talsperre,0,0,0,0,1,1,1
218,219,HWN219 Wippertalsperre,51.569720,11.192660,300,Wippertalsperre Bank am Rundweg,Wippertalsperre Bank am Rundweg,0,0,0,0,0,0,0
219,220,HWN220 Phillippsgruß,51.609504,10.531842,464,Phillippsgruß,Phillippsgruß,0,0,0,0,0,0,0
220,221,HWN221 Jungfernklippe,51.804200,10.513817,650,Jungfernklippe,Jungfernklippe,0,0,0,0,1,1,1


In [ ]:
print(f"All images of boxes: {all_stamps['box_image'].sum()}")
print(f"All images of selfies: {all_stamps['selfie_image'].sum()}")

In [ ]:
all_stamps.to_excel( 
                    Path(path_to_source, "excel", 'all_stamps_visited.xlsx'), index=False)